## Análise de sentimentos com algoritmo Naive Bayes

Este código demonstra a aplicação do algoritmo Naive Bayes em um projeto de análise de sentimentos. Foram utilizadas as bibliotecas pandas, numpy, tabulate, sklearn (CountVectorizer e MultinomialNB) e joblib para persistir o modelo treinado. O objetivo é classificar comentários de produtos como positivos ou negativos com base em sua pontuação geral e no texto do comentário, sendo comentários com nota maior ou igual a 3 considerados positivos, e abaixo disso negativos.

O algoritmo Naive Bayes é um método de classificação probabilístico simples, baseado no teorema de Bayes, com uma suposição "ingênua" de independência entre os recursos. Ele é frequentemente utilizado em problemas de análise de sentimentos e classificação de textos. Na prática, o algoritmo calcula a probabilidade de um comentário ser positivo ou negativo com base nas palavras presentes nele, assumindo que a presença de cada palavra é independente das outras (daí o "ingênuo").

O código carrega um conjunto de dados de avaliações de produtos, remove linhas com dados ausentes, cria uma nova coluna de classificação com base na pontuação geral e nos separa os comentários e suas classes. Em seguida, ele utiliza a classe CountVectorizer para converter os comentários em vetores de contagem de palavras. A vetorização de palavras é o processo de transformar textos em representações numéricas que possam ser usadas por algoritmos de aprendizado de máquina. No contexto do Naive Bayes para análise de sentimentos, a vetorização de palavras geralmente envolve a criação de um vocabulário com todas as palavras únicas nos textos e, em seguida, a contagem de quantas vezes cada palavra aparece em cada texto. Isso resulta em uma matriz em que cada linha representa um texto e cada coluna representa uma palavra do vocabulário, com os valores indicando a frequência de cada palavra no texto.

Os termos frequentes são avaliados com base em sua presença e frequência nos textos. No contexto da análise de sentimentos, palavras que aparecem com frequência em comentários positivos podem ser consideradas como indicativas de sentimentos positivos, enquanto palavras que aparecem com frequência em comentários negativos podem ser consideradas como indicativas de sentimentos negativos. O Naive Bayes utiliza essas frequências para calcular a probabilidade de um texto pertencer a uma determinada classe (positiva ou negativa), levando em conta a frequência de cada palavra no texto e a frequência das palavras em cada classe.

Em seguida, os vetores de contagem de palavras são usados para treinar um modelo Multinomial Naive Bayes. Esse modelo é então salvo em um arquivo para uso futuro.

Para testar o modelo, são fornecidos alguns comentários de teste que são transformados em vetores de contagem de palavras e passados para o modelo carregado, que faz previsões sobre se os comentários são positivos ou negativos. O resultado das previsões é então exibido no console.

In [2]:
# Imports necessários
import re
import string
import unicodedata
import numpy as np
import pandas as pd
from tabulate import tabulate
from joblib import dump, load
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Baixa recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Carrega o conjunto de dados
dataset = pd.read_csv("C:\\fatec\\B2W-Reviews01\\B2W-Reviews01.csv", usecols=['overall_rating', 'review_text'])

# Mostra o tamanho do dataset original
print("Tamanho do dataset original:", dataset.shape[0], "\n")

# Remove linhas com valores em branco
dataset = dataset.dropna(subset=['review_text', 'overall_rating'])

# Mostra o tamanho do dataset alterado
print("Tamanho do dataset alterado:", dataset.shape[0], "\n")


Tamanho do dataset original: 132373 

Tamanho do dataset alterado: 129098 



In [4]:
# Cria uma nova coluna para classificar entre comentários positivos, negativos ou neutros com base na nota:
dataset['feeling'] = np.where(dataset['overall_rating'] < 3, 'Negative', np.where(dataset['overall_rating'] == 3, 'Neutral', 'Positive'))

# Mostra os primeiros registros do dataset
print("Primeiros registros do dataset:")
print(tabulate(dataset.head(20), headers='keys', tablefmt='pipe'))


Primeiros registros do dataset:
|    |   overall_rating | review_text                                                                                                                                                                                                                                                         | feeling   |
|---:|-----------------:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------|
|  0 |                4 | Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.                                                                                                      | Positive  |
|  1 |                4 | Por apenas R$1994.20,eu consegui comprar e

In [5]:
# Pré-processamento dos dados
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

preprocessed_texts = []  # Lista para armazenar os textos pré-processados

# Itera sobre cada texto no dataset para pré-processamento
for text in dataset['review_text']:
    # Converter para minúsculas
    text = text.lower()
    # Remover acentos
    text = ''.join(char for char in unicodedata.normalize('NFKD', text) if unicodedata.category(char) != 'Mn')
    # Remover números usando expressão regular
    text = re.sub(r'\d+', '', text)
    # Remover caracteres especiais (incluindo emojis)
    text = re.sub(r'[^\w\s]', '', text)
    # Remover pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenização
    tokens = word_tokenize(text)
    # Lematização e remoção de stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word.lower() not in stop_words]
    # Juntar tokens em texto novamente
    preprocessed_text = ' '.join(tokens)
    preprocessed_texts.append(preprocessed_text)  # Adicionar texto pré-processado à lista

# Atualiza o DataFrame com os textos pré-processados
dataset['review_text'] = preprocessed_texts

# Mostra o dataset após o pré-processamento
print("\nDataset após pré-processamento:")
print(tabulate(dataset.head(20), headers='keys', tablefmt='pipe'))



Dataset após pré-processamento:
|    |   overall_rating | review_text                                                                                                                                                                                        | feeling   |
|---:|-----------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------|
|  0 |                4 | contente compra entrega rapida unico problema americana troca devolucao produto consumidor problemas espera                                                                                        | Positive  |
|  1 |                4 | apenas reu consegui comprar lindo copo acrilico                                                                                                                                                    | Positive  |
|  2 |                4 | s

In [6]:
# Separar os dados em features (X) e target (Y)
X = dataset['review_text'].values
Y = dataset['feeling'].values

# Criar um objeto CountVectorizer com N-grams
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3))  # Considera unigramas, bigramas e trigramas

# Vetorizar os dados de texto com N-grams
X_ngrams = ngram_vectorizer.fit_transform(X)

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, Y_train, Y_test = train_test_split(X_ngrams, Y, test_size=0.3, random_state=42)

# Criar e treinar o modelo Naive Bayes com os dados vetorizados com N-grams
model_ngrams = MultinomialNB()
model_ngrams.fit(X_train, Y_train)

# Avaliar o modelo com N-grams
accuracy_ngrams = model_ngrams.score(X_test, Y_test)
print("Acurácia:", accuracy_ngrams)


Acurácia: 0.8240898528272657


In [7]:
# Prever as classes para os dados de teste
Y_pred = model_ngrams.predict(X_test)

# Criar um DataFrame com as previsões e as verdadeiras classes de "feeling"
df_results = pd.DataFrame({'Feeling_Predicted': Y_pred, 'Feeling_True': Y_test})

# Mostrar os resultados
print("\nResultados de 'feeling' após o treinamento do modelo:")
print(df_results.head(20))  # Exibir os 20 primeiros registros



Resultados de 'feeling' após o treinamento do modelo:
   Feeling_Predicted Feeling_True
0           Positive     Positive
1           Positive     Positive
2           Positive      Neutral
3           Negative     Negative
4           Negative     Positive
5           Positive     Positive
6           Positive     Negative
7           Positive      Neutral
8           Positive     Positive
9           Positive     Positive
10          Positive     Positive
11          Positive     Positive
12          Positive     Positive
13          Negative     Negative
14          Positive     Positive
15          Positive     Positive
16          Positive     Positive
17          Positive     Positive
18          Positive      Neutral
19          Positive     Positive


In [8]:
# Salvar o modelo e o vetorizador em um único arquivo, no mesmo diretório deste notebook
dump((model_ngrams), 'modelo_e_vetorizador.joblib')

# Carregar o modelo e o vetorizador do arquivo
modelo_carregado = load('modelo_e_vetorizador.joblib')
